In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import torchvision
from Kick import kick2gpu

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout2d(p=0.25)
        self.dropout2 = nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(in_features=64*12*12, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1e-3)

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])), batch_size=10, shuffle=True)
dataiter = iter(train_loader)

In [5]:
@kick2gpu
def run():
    for batch_idx, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(X)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print("loss at epoch", batch_idx, loss.item())

>> initialize


In [6]:
run()

>>  torch.optim found
>>  torchvision found
>>  torch.nn found
>>  torch.nn.functional found
>>  numpy found
>>  torch found
b'loss at epoch 0 2.2899022102355957'
b'loss at epoch 10 2.262838125228882'
b'loss at epoch 20 2.3278121948242188'
b'loss at epoch 30 2.373546600341797'
b'loss at epoch 40 2.2993247509002686'
b'loss at epoch 50 2.3009605407714844'
b'loss at epoch 60 2.3721916675567627'
b'loss at epoch 70 2.2717480659484863'
b'loss at epoch 80 2.273516893386841'
b'loss at epoch 90 2.2721245288848877'
b'loss at epoch 100 2.2755868434906006'
b'loss at epoch 110 2.2977397441864014'
b'loss at epoch 120 2.2520251274108887'
b'loss at epoch 130 2.2681915760040283'
b'loss at epoch 140 2.248382091522217'
b'loss at epoch 150 2.2398502826690674'
b'loss at epoch 160 2.280900716781616'
b'loss at epoch 170 2.284872531890869'
b'loss at epoch 180 2.2812535762786865'
b'loss at epoch 190 2.307100296020508'
b'loss at epoch 200 2.2640795707702637'
b'loss at epoch 210 2.260305881500244'
b'loss at epoc